# WESAD PyTorch Training - Google Colab

Este notebook treina modelos PyTorch para classificação binária de stress usando o dataset WESAD.

## Características:
- ✅ Modelo CNN-LSTM otimizado para dados fisiológicos
- ✅ Treinamento baseline e com Differential Privacy (DP)
- ✅ Comparação de resultados entre técnicas
- ✅ Progress bar com ETA durante treinamento
- ✅ Resultados reproduzíveis com seeds fixos
- ✅ Métricas específicas para classificação binária

## Requisitos:
- Google Colab com GPU habilitada
- Dados no Google Drive: `mydrive/mhealth-data/data/processed/wesad/`


## 1. Configuração Inicial


In [ ]:
# Instalar dependências necessárias
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy pandas scikit-learn matplotlib seaborn
!pip install opacus  # Para Differential Privacy

# Clonar o repositório
!git clone https://github.com/vasco-fernandes21/mhealth-data-privacy.git
import sys
sys.path.append('/content/mhealth-data-privacy')

print("✅ Dependências instaladas e repositório clonado")


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verificar se os dados existem
import os
data_path = '/content/drive/MyDrive/mhealth-data/data/processed/wesad'

if os.path.exists(data_path):
    print(f"✅ Dados encontrados em: {data_path}")

    # Listar arquivos
    files = os.listdir(data_path)
    print(f"📁 Arquivos disponíveis: {files}")

    # Verificar tamanhos
    for file in ['X_train.npy', 'y_train.npy', 'X_val.npy', 'y_val.npy', 'X_test.npy', 'y_test.npy']:
        if file in files:
            size = os.path.getsize(os.path.join(data_path, file))
            print(f"  {file}: {size / (1024*1024):.1f} MB")
        else:
            print(f"  ❌ {file} não encontrado")
else:
    print(f"❌ Dados não encontrados em: {data_path}")
    print("💡 Certifique-se de que os dados estão no caminho correto no Google Drive")


## 2. Treinamento dos Modelos


In [ ]:
# Executar treinamento baseline WESAD
import os, shutil

repo_data_dir = '/content/mhealth-data-privacy/data/processed/wesad'
drive_data_dir = '/content/drive/MyDrive/mhealth-data/data/processed/wesad'
baseline_models_dir = '/content/mhealth-data-privacy/models/wesad/baseline'
baseline_results_dir = '/content/mhealth-data-privacy/results/wesad/baseline'
dp_models_dir = '/content/mhealth-data-privacy/models/wesad/differential_privacy'
dp_results_dir = '/content/mhealth-data-privacy/results/wesad/differential_privacy'

# Garantir diretórios de saída
os.makedirs(baseline_models_dir, exist_ok=True)
os.makedirs(baseline_results_dir, exist_ok=True)
os.makedirs(dp_models_dir, exist_ok=True)
os.makedirs(dp_results_dir, exist_ok=True)

# Criar ligação simbólica dos dados
os.makedirs('/content/mhealth-data-privacy/data/processed', exist_ok=True)
if os.path.islink(repo_data_dir) or os.path.exists(repo_data_dir):
    try:
        if os.path.islink(repo_data_dir):
            os.unlink(repo_data_dir)
        else:
            shutil.rmtree(repo_data_dir)
    except Exception as e:
        print(f"Aviso ao remover destino antigo: {e}")

# Criar symlink
!ln -sf "$drive_data_dir" "$repo_data_dir"
print(f"✅ Dados referenciados via symlink: {repo_data_dir} -> {drive_data_dir}")

print(f"🚀 Iniciando treinamento com dados de: {drive_data_dir}")
print("=" * 80)

# 1. Treinar Baseline
print("🎯 TREINANDO BASELINE WESAD...")
print("-" * 50)
!python /content/mhealth-data-privacy/src/train/wesad/train_baseline.py

print("\n" + "=" * 80)
print("🎯 TREINANDO WESAD COM DIFFERENTIAL PRIVACY...")
print("-" * 50)
!python /content/mhealth-data-privacy/src/train/wesad/differential_privacy/train_dp.py

print("✅ Treinamentos concluídos!")


## 3. Análise Comparativa dos Resultados


In [ ]:
# Carregar e comparar os resultados
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Carregar resultados
baseline_results_path = '/content/mhealth-data-privacy/models/wesad/baseline/results_wesad.json'
dp_results_path = '/content/mhealth-data-privacy/models/wesad/differential_privacy/results_wesad_dp.json'

baseline_results = None
dp_results = None

if os.path.exists(baseline_results_path):
    with open(baseline_results_path, 'r') as f:
        baseline_results = json.load(f)
    print("✅ Resultados baseline carregados")
else:
    print(f"❌ Resultados baseline não encontrados em: {baseline_results_path}")

if os.path.exists(dp_results_path):
    with open(dp_results_path, 'r') as f:
        dp_results = json.load(f)
    print("✅ Resultados DP carregados")
else:
    print(f"❌ Resultados DP não encontrados em: {dp_results_path}")

if baseline_results and dp_results:
    print("\n📊 COMPARAÇÃO DE RESULTADOS:")
    print("=" * 70)
    
    # Tabela comparativa
    comparison_data = {
        'Métrica': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
        'Baseline': [
            f"{baseline_results['accuracy']:.4f}",
            f"{baseline_results['precision']:.4f}",
            f"{baseline_results['recall']:.4f}",
            f"{baseline_results['f1_score']:.4f}"
        ],
        'Com DP': [
            f"{dp_results['accuracy']:.4f}",
            f"{dp_results['precision']:.4f}",
            f"{dp_results['recall']:.4f}",
            f"{dp_results['f1_score']:.4f}"
        ],
        'Diferença': [
            f"{dp_results['accuracy'] - baseline_results['accuracy']:+.4f}",
            f"{dp_results['precision'] - baseline_results['precision']:+.4f}",
            f"{dp_results['recall'] - baseline_results['recall']:+.4f}",
            f"{dp_results['f1_score'] - baseline_results['f1_score']:+.4f}"
        ]
    }
    
    df = pd.DataFrame(comparison_data)
    print(df.to_string(index=False))
    
    # Privacy budget
    if 'dp_params' in dp_results:
        print(f"\n🔒 PRIVACY BUDGET (DP): ε = {dp_results['dp_params']['epsilon']:.2f}")
    
    # Matrizes de confusão lado a lado
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Baseline confusion matrix
    cm_baseline = np.array(baseline_results['confusion_matrix'])
    sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', 
                xticklabels=baseline_results['class_names'], 
                yticklabels=baseline_results['class_names'], ax=ax1)
    ax1.set_title(f'Baseline - Accuracy: {baseline_results["accuracy"]:.3f}')
    ax1.set_xlabel('Predito')
    ax1.set_ylabel('Real')
    
    # DP confusion matrix
    cm_dp = np.array(dp_results['confusion_matrix'])
    sns.heatmap(cm_dp, annot=True, fmt='d', cmap='Reds', 
                xticklabels=dp_results['class_names'], 
                yticklabels=dp_results['class_names'], ax=ax2)
    ax2.set_title(f'Com DP - Accuracy: {dp_results["accuracy"]:.3f}')
    ax2.set_xlabel('Predito')
    ax2.set_ylabel('Real')
    
    plt.tight_layout()
    plt.show()
    
    # Bar plot comparison
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    baseline_scores = [baseline_results['accuracy'], baseline_results['precision'], 
                      baseline_results['recall'], baseline_results['f1_score']]
    dp_scores = [dp_results['accuracy'], dp_results['precision'], 
                dp_results['recall'], dp_results['f1_score']]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(x - width/2, baseline_scores, width, label='Baseline', color='skyblue')
    ax.bar(x + width/2, dp_scores, width, label='Com DP', color='salmon')
    
    ax.set_xlabel('Métricas')
    ax.set_ylabel('Score')
    ax.set_title('Comparação Baseline vs Differential Privacy')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.set_ylim(0, 1)
    
    # Add value labels on bars
    for i, v in enumerate(baseline_scores):
        ax.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    for i, v in enumerate(dp_scores):
        ax.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    plt.show()
    
else:
    print("💡 Execute primeiro a célula de treinamento para gerar os resultados")


## 💡 Dicas e Troubleshooting

### Problemas Comuns:

1. **Dados não encontrados:**
   - Verifique se o caminho `mydrive/mhealth-data/data/processed/wesad/` está correto
   - Certifique-se de que todos os arquivos `.npy` estão presentes

2. **GPU não disponível:**
   - Vá em Runtime → Change runtime type → Hardware accelerator → GPU
   - Os modelos funcionarão na CPU, mas serão mais lentos

3. **Memória insuficiente:**
   - O dataset WESAD é menor que o Sleep-EDF (~200MB)
   - Geralmente não há problemas de memória no Colab

4. **Differential Privacy não converge:**
   - É normal que modelos DP tenham performance inferior
   - O importante é o trade-off privacidade vs utilidade
   - Ajuste hiperparâmetros DP se necessário

### Características do WESAD:
- **Classes**: 2 (non-stress, stress) - classificação binária
- **Dados fisiológicos**: ECG, EDA, EMG, RESP, TEMP, ACC
- **Janelas temporais**: 1920 samples por janela
- **Distribuição**: ~70% non-stress, ~30% stress

### Comparação com Sleep-EDF:
- **Sleep-EDF**: 5 classes (estágios de sono), dados mais complexos
- **WESAD**: 2 classes (stress binário), dados fisiológicos em tempo real
- **Baseline WESAD**: ~97% accuracy (muito alto devido à distribuição)
- **Baseline Sleep-EDF**: ~85% accuracy (mais desafiador)

### Interpretação dos Resultados DP:
- **Privacy Budget (ε)**: Valores menores = mais privacidade
- **Trade-off**: Menos privacidade = melhor accuracy
- **Configuração atual**: ε ≈ 20-30 (privacidade moderada)

---

**Notebook criado para o projeto Mestrado SIDM - MHealth Data Privacy** 🚀
